# Install dependancies

Rendering Dependancies



In [0]:
#remove " > /dev/null 2>&1" to see what is going on under the hood
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

Gym Dependancies

In [0]:
!apt-get update > /dev/null 2>&1
!apt-get install cmake > /dev/null 2>&1
!pip install --upgrade setuptools 2>&1
!pip install ez_setup > /dev/null 2>&1
!pip install gym[atari] > /dev/null 2>&1

# Imports and Helper functions


In [0]:
import gym
from gym import logger as gymlogger
from gym.wrappers import Monitor
gymlogger.set_level(40) #error only
import tensorflow as tf
import numpy as np
import random
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import glob
import io
import base64
from IPython.display import HTML

from IPython import display as ipythondisplay
from IPython.display import clear_output

import pickle
import torch
import torch as T
import torch.nn as nn
import torch.nn.functional as F
import copy
import time
from skimage.transform import downscale_local_mean


In [0]:
"""
Utility functions to enable video recording of gym environment and displaying it
To enable video, just do "env = wrap_env(env)""
"""
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
      mp4 = mp4list[0]
      video = io.open(mp4, 'r+b').read()
      encoded = base64.b64encode(video)
      ipythondisplay.display(HTML(data='''<video alt="test" autoplay 
                  loop controls style="height: 400px;">
                  <source src="data:video/mp4;base64,{0}" type="video/mp4" />
              </video>'''.format(encoded.decode('ascii'))))
    else: 
      print("Could not find video")
      

def wrap_env(env):
  env = Monitor(env, './video', force=True)
  return env

# Program

Neural Network

In [0]:
class Q_Network(nn.Module):
    def __init__(self):
        super(Q_Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc1 = nn.Linear(4 * 64 * 7 * 7, 512)   # [(W-K+2P)/S]+1
        self.fc2 = nn.Linear(512, 6)

        self.device = T.device('cuda')
        self.to(self.device)
        
    def forward(self, batch):
        
        flatten = torch.Tensor()
        
        f0 = batch[:, 0, :, :].view(-1, 1, 84,84)
        f1 = batch[:, 1, :, :].view(-1, 1, 84,84)
        f2 = batch[:, 2, :, :].view(-1, 1, 84,84)
        f3 = batch[:, 3, :, :].view(-1, 1, 84,84)
        
        f0 = F.relu(self.conv1(f0))
        f1 = F.relu(self.conv1(f1))
        f2 = F.relu(self.conv1(f2))
        f3 = F.relu(self.conv1(f3))
        
        f0 = F.relu(self.conv2(f0))
        f1 = F.relu(self.conv2(f1))
        f2 = F.relu(self.conv2(f2))
        f3 = F.relu(self.conv2(f3))
        
        f0 = F.relu(self.conv3(f0))
        f1 = F.relu(self.conv3(f1))
        f2 = F.relu(self.conv3(f2))
        f3 = F.relu(self.conv3(f3))
        
        f0 = f0.view(-1, 64 * 7 * 7)
        f1 = f1.view(-1, 64 * 7 * 7)
        f2 = f2.view(-1, 64 * 7 * 7)
        f3 = f3.view(-1, 64 * 7 * 7)
        flatten = torch.cat( (f0, f1, f2, f3), dim=1 )
        
        fc = F.relu(self.fc1(flatten))
        action_values = self.fc2(fc)
        
        return action_values    # [Q action_values] * frames

Replay Buffer

In [0]:
class ReplayBuffer(object):
    def __init__(self):
        self.buffer = []
        self.max_length = 50000

    def push(self, data):
        self.buffer.append( data )
        if len(self.buffer) > self.max_length:
          del self.buffer[0]
        
    def sample(self, batch_size):
        minibatch = random.sample(self.buffer, batch_size)

        states = np.array([i[0] for i in minibatch]).astype('float32')
        next_states = np.array([i[1] for i in minibatch]).astype('float32')
        actions = np.array([i[2] for i in minibatch]).astype('float32')
        rewards = np.array([i[3] for i in minibatch]).astype('float32')
        terminals = np.array([i[4] for i in minibatch]).astype('float32')
        
        return  T.from_numpy(states).cuda(), \
                T.from_numpy(next_states).cuda(), \
                T.from_numpy(actions).cuda(), \
                T.from_numpy(rewards).cuda(), \
                T.from_numpy(terminals).cuda()

Agent

In [0]:
class DQNAgent(object):
    def __init__(self, model, optim):
        self.model = model
        self.target = copy.deepcopy(self.model)
        self.optim = optim
        self.gamma = 0.99
        self.huber_loss = nn.SmoothL1Loss()
        self.replay_buffer = ReplayBuffer()
        self.tau = 0.05
        pass
    
    def update(self):
        for params, target_params in zip(self.model.parameters(), self.target.parameters()):
            target_params.data.copy_(params.data * self.tau + target_params.data * (1 - self.tau))
    
    def select_action(self, state, epsilon):
        if np.random.uniform(0, 1) < epsilon:
            return env.action_space.sample()  # choose random action
        else:
            t_state = T.from_numpy(state[np.newaxis,:,:,:]).cuda()
            qvalues = self.model(t_state.float()).cuda()
            qvalues = qvalues.cpu().data.numpy()
            return np.argmax(qvalues)  # choose greedy action

    def learn(self, batch_size):
        
        if(len(self.replay_buffer.buffer) < batch_size):
            return
        
        states, next_states, actions, rewards, dones = self.replay_buffer.sample(batch_size)
        
        state_qvalues = T.gather(self.model(states).cuda(), dim=1, index=actions.long()).cuda()
        next_state_qvalues = self.target(next_states).cuda()
        next_actions, _ = T.max(next_state_qvalues, dim=1, keepdim=True)
        not_dones = 1 - dones
        target_values = rewards + not_dones * self.gamma * next_actions
        loss = self.huber_loss(state_qvalues, target_values.detach())
        self.optim.zero_grad()
        loss.backward()
        self.optim.step()

Helper Funtions

In [0]:
def preprocess(frame):
    r = frame[34:194,:,0]
    g = frame[34:194,:,1]
    b = frame[34:194,:,2]
    frame = 0.299 * r + 0.587 * g + 0.114 * b
    frame = np.pad(frame, (4,4), mode='edge')
    frame = downscale_local_mean(frame, (2, 2))
    
    frame = frame[np.newaxis,:,:]
    
    return frame
 
def generate_test_set(env):
        
    done = False
    frame = preprocess(env.reset())
    state = np.repeat(frame, 4, axis=0)
    states = []
    
    while not done:
        frame, reward, done, _ = env.step(env.action_space.sample())
        
        frame = preprocess(frame)
        next_state = np.delete(state, 0, 0)
        next_state = np.concatenate((next_state, frame), axis=0)
        
        states.append(next_state)
        state = next_state
        
    return torch.tensor(states, device=cuda).float()

def eval_model(data, model):
    predictions = model(data)
    return predictions.cpu().data.numpy()

GPU Setup

In [0]:
device_number = T.cuda.current_device()
print('Cuda avaibale:', T.cuda.is_available())
print('Cuda devices:', T.cuda.device_count())
print('Name:', T.cuda.get_device_name(device_number))

cuda = T.device('cuda')


Link Google Drive

In [0]:
#from google.colab import drive
#drive.mount('/content/gdrive/')

In [0]:
#!ls '/content/gdrive/My Drive/Models'

Main Loop

In [0]:
env = gym.make("PongDeterministic-v4")

state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n

model = Q_Network()
optimizer = T.optim.Adam( model.parameters(), lr=0.0001)
    
agent = DQNAgent(model, optimizer)

episode = 0
step = 0
plotting = False 

max_time_steps = 5000000
batch_size = 32
epsilon_decay = 0.9 / 500000
epsilon = 1 - (step * epsilon_decay)
epsilon_min = 0.1

rewards = []
averages = []
epsilon_history = []
step_history = []

while step < max_time_steps:
    print("\nEpisode: {ep:3d} - Epsilon: {eps:.7f}".format(ep=episode, eps=epsilon))
    
    reward_sum = 0
    frame = preprocess(env.reset())
    state = np.repeat(frame, 4, axis=0)
    terminal = False
    step_history.append(step)
    
    while not terminal:
        
        action = agent.select_action(state , epsilon)
        frame, reward, terminal, _ = env.step(action)
        
        frame = preprocess(frame)
        next_state = np.delete(state, 0, 0)
        next_state = np.concatenate((next_state, frame), axis=0)
        
        reward_sum += reward
        
        agent.replay_buffer.push( (state, next_state, [action], [reward], [terminal]) )
        state = next_state
            
        agent.learn(batch_size)
        if step % 32 == 0:
            agent.update()
        
        step += 1

        if step % 25000 == 0:
          plotting = True

        if epsilon > epsilon_min:
          epsilon -= epsilon_decay 
        
    rewards.append(reward_sum)
    average = np.mean(rewards[-100:])
    averages.append(average)
    epsilon_history.append(epsilon)

    print("Average over last 100 episode: {0:.2f}".format(average))
    print("Reward =", reward_sum)
    print("           ---- Steps =", step, "----\n")

    if plotting:
        plotting = False
        fig, ax1 = plt.subplots()
        ax1.plot(step_history, rewards, label = "Rewards")
        ax1.plot(step_history, averages, label = "Averages")
        ax2 = ax1.twinx()
        ax2.plot(step_history, epsilon_history, label = "Epsilon", color = 'red')
        ax2.set_ylabel('Epsilon')
        ax1.set_xlabel('Network Updates')
        plt.show()
        print("\n         -------------------------------     \n")

    episode += 1

env.close()